In [1]:
from bs4 import BeautifulSoup
from dateutil.parser import parse

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities


firefox_capabilities = DesiredCapabilities.FIREFOX
firefox_capabilities['marionette'] = True
firefox_capabilities['handleAlerts'] = True
firefox_capabilities['acceptSslCerts'] = True
firefox_capabilities['acceptInsecureCerts'] = True
geckoPath = 'driver/geckodriver.exe'

firefox = webdriver.Firefox(capabilities=firefox_capabilities, executable_path=geckoPath)
driver = webdriver.PhantomJS(executable_path='driver/phantomjs.exe')


#firefox.get('https://www.crunchbase.com/organization/apple')
driver.get('https://techcrunch.com/')
firefox.set_page_load_timeout(3)
driver.set_page_load_timeout(30)
def extract_company_techcrunch_2drivers(company):
    base_url = 'www.crunchbase.com'
    url = 'https://techcrunch.com/search/'+company
    try:
        driver.get(url)
    except:
        print('timeout of phantomjs')
        pass
    soup = BeautifulSoup(driver.page_source, "html.parser")
    blocks = soup.select("h2.post-title a")
    founded = None
    employees = None
    if len(blocks)>0:
        url_crunchbase = blocks[0]["href"]  
        name  = blocks[0].getText()
 
        if base_url in url_crunchbase:
            print(company,' in crunch base')
            try:
                firefox.get(url_crunchbase)
            except:
                pass
            
            blocks_dt = []
            blocks_dd = []
            try:
                soup = BeautifulSoup(firefox.page_source, "html.parser")
                blocks_dt = soup.select("div.details dt")
                blocks_dd = soup.select("div.details dd")
            except:
                pass
            
            i = 0
            while len(blocks_dt)==0 or len(blocks_dd)==0:
                i=i+1
                if i >= 50:
                    print('fail to crawl ', company , ' in crunchbase')
                    break
                time.sleep(0.3)
                try:
                    soup = BeautifulSoup(firefox.page_source, "html.parser")
                    blocks_dt = soup.select("div.details dt")
                    blocks_dd = soup.select("div.details dd")
                except:
                    pass
                  
            for index,block in enumerate(blocks_dt):
                if 'Founded' in  block.getText() : 
                    founded = blocks_dd[index].getText()
                if 'Employees' in block.getText():
                    employees = blocks_dd[index].getText().split('|')[0]
            print(company,founded,employees)
    return founded,employees

In [2]:
import time
import json
import os

class CompaniesManager:
    def __init__(self):
        self.companies = []
        self.articles = []
        self.companies_name = []
        self.load_articles()
        self.load_companies()
        
    def load_articles(self):
        if os.path.isfile('data/raw_articles.json'):
            f = open('data/raw_articles.json')
            self.articles = json.load(f)
            f.close()
            
    def load_companies(self):
        if os.path.isfile('data/raw_companies.json'):
            f = open('data/raw_companies.json')
            self.companies = json.load(f)
            f.close()      
            for company in self.companies:
                name = company["name"]
                self.companies_name.append(name)
        
            
    def extract_companies(self):
        
        for index,article in enumerate(self.articles):
            for company_name in article["companies"]: 
                article_extraInfos = article["extra_infos"]
                for info in article_extraInfos:
                    if info["text"]== company_name:
                        relevance = info["relevance"]
                        count_in_article = info["count"]
                            
                if company_name in self.companies_name :
                    for company in self.companies:
                        if company_name == company["name"] and article["id"] not in company["articles"]: 
                            company["count"] = company["count"]+1
                            company["sentiment"]= company["sentiment"]+article["sentiment"]
                            company["articles"].append(article["id"])
                            extra_infos = {
                                 "id":article["id"] ,
                                 "count_in_article":count_in_article,
                                 "revelance": relevance       
                            }
                            company["extra_infos"].append(extra_infos)
                            
                else:
                    company = {
                        "name": company_name,
                        "dateFound": int(str(time.time()).split('.')[0]),
                        "count":1,
                        "sentiment": article["sentiment"],
                        "articles": [article["id"]],
                        "extra_infos":
                        [
                            {
                             "id": article["id"],
                             "count_in_article":count_in_article,
                             "revelance": relevance
                            }  
                        ]    
                    }
                    self.companies.append(company)
                    self.companies_name.append(company_name)
            
        self.save_to_disk()
        
    def extend_crunch(self):
        
        i = 0
        for index,company in enumerate(self.companies):
            name = company["name"]
                
            if 'foundationDate' not in company and 'number_of_employees' not in company and "search_label" not in company.keys():
                i=i+1
                founded,employees = extract_company_techcrunch_2drivers(name)
                company["search_label"] = 1
                if founded is not None:
                    company["foundationDate"] = founded
                if employees is not None:
                    company["number_of_employees"] = employees
                    
            elif 'foundationDate'  in company or 'number_of_employees' in company:
                company["search_label"] = 1
                print("skip: ",name, " in crunchbase")
            
            else:
                print("skip: ",name, " not in crunchbase")
            
            if (index+1)%200 == 0:
                self.save_to_disk()
        self.save_to_disk()
                    
            
                
            
                                
    def save_to_disk(self):
        with open('data/raw_companies.json', 'w') as company_file:
            json.dump(self.companies, company_file,indent = 2)
            company_file.close()
                
                
                
            
    

In [3]:
CM = CompaniesManager()
#CM.extract_companies()
CM.extend_crunch()

skip:  SENSORO  in crunchbase
skip:  Microsoft Corp  not in crunchbase
skip:  Sumitomo  not in crunchbase
skip:  Manchester City Verve  not in crunchbase
skip:  SalesWings  in crunchbase
skip:  SA tech  not in crunchbase
skip:  Avito  not in crunchbase
skip:  Tshimologong Innovation Hub  not in crunchbase
skip:  Flickr  in crunchbase
skip:  Niantic Labs  not in crunchbase
skip:  Oblong  not in crunchbase
skip:  Osterhout Design Group  not in crunchbase
skip:  Niantic  not in crunchbase
skip:  Oblong Industries  in crunchbase
skip:  Mezzanine  not in crunchbase
skip:  The Pokemon Company  in crunchbase
skip:  GamesBeat  in crunchbase
skip:  BNJ Marketing  not in crunchbase
skip:  DormaKaba  not in crunchbase
skip:  Berkshire companies  not in crunchbase
skip:  Bezos  not in crunchbase
skip:  Private Equity  not in crunchbase
skip:  Utrip  in crunchbase
skip:  Greylock  not in crunchbase
skip:  Elsight  not in crunchbase
skip:  LiveU  in crunchbase
skip:  WAN  not in crunchbase
skip:  Vo

skip:  XL Travel  not in crunchbase
skip:  FOMO  in crunchbase
skip:  GOAP  not in crunchbase
skip:  CcHUB  not in crunchbase
skip:  Strauss Energy  in crunchbase
skip:  BIPV Stima  not in crunchbase
skip:  Jumia  in crunchbase
skip:  Picolino  not in crunchbase
skip:  The Hershey Company  in crunchbase
skip:  Webber Wentzel  not in crunchbase
skip:  Michalsons  not in crunchbase
skip:  Pathao  in crunchbase
skip:  Go-Jek  in crunchbase
skip:  Rocket Internet  in crunchbase
skip:  Evernote  in crunchbase
skip:  MobileDay  in crunchbase
skip:  Camcard  not in crunchbase
skip:  M4JAM  not in crunchbase
skip:  Informal Solutions  not in crunchbase
skip:  Click2Sure  not in crunchbase
skip:  Guasco  not in crunchbase
skip:  Stellenbosch Lean Coffee  not in crunchbase
skip:  MEST Africa  not in crunchbase
skip:  Polymorph Offices  not in crunchbase
skip:  WeFix  in crunchbase
skip:  BitX  not in crunchbase
skip:  iKhokha  not in crunchbase
skip:  SweepSouth  in crunchbase
skip:  LifeQ  in c

skip:  AB InBev  not in crunchbase
skip:  toys-to-life  not in crunchbase
skip:  Animoca Brands  not in crunchbase
skip:  PowerCore  in crunchbase
skip:  Toys for Bob  not in crunchbase
skip:  Statista  in crunchbase
skip:  DesignerCon  not in crunchbase
skip:  PuzzleCore  not in crunchbase
skip:  Skylanders  not in crunchbase
skip:  HP Inc  not in crunchbase
skip:  CryptoStopper  not in crunchbase
skip:  Cloud Security  not in crunchbase
skip:  WatchPoint  not in crunchbase
skip:  Quickbooks  not in crunchbase
skip:  Templafy  in crunchbase
skip:  Trello  in crunchbase
skip:  EFF  not in crunchbase
skip:  Wickr Pro  not in crunchbase
skip:  CyberSponse  in crunchbase
skip:  Legal Shield  not in crunchbase
skip:  GetResponse  in crunchbase
skip:  Jigsaw  in crunchbase
skip:  Research in Motion  not in crunchbase
skip:  Totsuko  not in crunchbase
skip:  Lemnos Labs  in crunchbase
skip:  Alumnify.  not in crunchbase
skip:  Etsy  in crunchbase
skip:  Faraday Future  in crunchbase
skip:  M

skip:  Siren  in crunchbase
skip:  NEW INC  in crunchbase
skip:  Match.com  not in crunchbase
skip:  Crelate  in crunchbase
skip:  Ascentium  not in crunchbase
skip:  Avanade  in crunchbase
skip:  Puzzazz  in crunchbase
skip:  Heptio  in crunchbase
skip:  Nohla Therapeutics  in crunchbase
skip:  Arivale  in crunchbase
skip:  InfoSpace  in crunchbase
skip:  Snowflake  not in crunchbase
skip:  Snowflake Computing  in crunchbase
skip:  Azure Data Warehouse  not in crunchbase
skip:  GigaOm  in crunchbase
skip:  Qumulo  in crunchbase
skip:  Isilon Systems  in crunchbase
skip:  Qumulo Core  not in crunchbase
skip:  Tyche Partners  in crunchbase
skip:  Sinclair Oil  in crunchbase
skip:  Ripl  in crunchbase
skip:  RealNetworks  not in crunchbase
skip:  Smilebox  in crunchbase
skip:  Kauffman  not in crunchbase
skip:  EveryMove  in crunchbase
skip:  higi  in crunchbase
skip:  High Fidelity  in crunchbase
skip:  Limitless  in crunchbase
skip:  Bungie  in crunchbase
skip:  ToyTalk  not in crunchb

skip:  Bondi Vet  not in crunchbase
skip:  Wink Models  not in crunchbase
skip:  Flatfish  in crunchbase
skip:  HealthyAZ  not in crunchbase
skip:  Tamarisic  not in crunchbase
skip:  Inlight Media  not in crunchbase
skip:  Eventfinda  in crunchbase
skip:  McGlinn  not in crunchbase
skip:  Virtual Content Group  not in crunchbase
skip:  Cow Prod  not in crunchbase
skip:  Worldviz  in crunchbase
skip:  World Innovation Lab  not in crunchbase
skip:  7-Eleven  in crunchbase
skip:  Flirtey  in crunchbase
skip:  Starship Technologies  in crunchbase
skip:  OpenView  in crunchbase
skip:  Airly  in crunchbase
skip:  McEncroe  not in crunchbase
skip:  Shebah  in crunchbase
skip:  Hyper  not in crunchbase
skip:  Australian Water Holdings  not in crunchbase
skip:  NIB Foundation  not in crunchbase
skip:  Vavven  not in crunchbase
skip:  SxSWedu  in crunchbase
skip:  Quizling  in crunchbase
skip:  Bonjoro  in crunchbase
skip:  USyd  not in crunchbase
skip:  Clinks  not in crunchbase
skip:  Happn  

skip:  Prodsmart  in crunchbase
skip:  FarmCloud  in crunchbase
skip:  Caixa Empreender Award  not in crunchbase
skip:  agricultures  not in crunchbase
skip:  YuScale  not in crunchbase
skip:  Benefit Cosmetics  in crunchbase
skip:  Hackerspace  in crunchbase
skip:  Pomantso  not in crunchbase
skip:  NUMA Barcelona  not in crunchbase
skip:  Vitcord  in crunchbase
skip:  Small Giant  not in crunchbase
skip:  First Fellow Partners  in crunchbase
skip:  CEE  in crunchbase
skip:  KickUP Sports Innovation  not in crunchbase
skip:  ScanTrust  in crunchbase
skip:  Biowatch  in crunchbase
skip:  Advanon  in crunchbase
skip:  Lunaphore Technologies  in crunchbase
skip:  Lunaphore  not in crunchbase
skip:  Comfylight  not in crunchbase
skip:  MUB cargo  in crunchbase
skip:  MUB  not in crunchbase
skip:  Fanmiles  in crunchbase
skip:  Auden AG  not in crunchbase
skip:  Ola Cafe  in crunchbase
skip:  Zomato Order  not in crunchbase
skip:  Foodpanda  in crunchbase
skip:  Flipkart  in crunchbase
ski

MedRhythms  in crunch base
MedRhythms 2013 1 - 10 
Safeguard  in crunch base
Safeguard October 1, 2013 None found in Crunchbase
Rippleshot  in crunch base
Rippleshot 2012 1 - 10 
Zulily  in crunch base
Zulily 2012 1 - 10 
Firstbird  in crunch base
Firstbird March 9, 2013 11 - 50 
MarketMuse  in crunch base
MarketMuse 2013 1 - 10 
MailChimp  in crunch base
MailChimp 2013 1 - 10 
Conversion XL  in crunch base
Conversion XL 2011 11 - 50 
Future Fifty  in crunch base
Future Fifty 2011 11 - 50 
Values Partnerships  in crunch base
Values Partnerships Unknown None
American Airlines  in crunch base
American Airlines Unknown None
Caribu  in crunch base
Caribu 2013 1 - 10 
Delta Gloves  in crunch base
fail to crawl  Delta Gloves  in crunchbase
Delta Gloves None None
PureCarbon  in crunch base
PureCarbon September 15, 2015 2 in Crunchbase
Consensys  in crunch base
Consensys September 15, 2015 2 in Crunchbase
Mattermark  in crunch base
Mattermark 2012 11 - 50 
Lionseye Group  in crunch base
Lionse

KeyboardInterrupt: 